# this notebook parse the **patch** data

In [ ]:
import numpy as np
import argparse

import json

import os
import math
from math import sin

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.offline as pyo
import plotly.graph_objs as go

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
from plotting_tools import *
from stage1_patch import *
from PatchParser import *
from TimeStampReader import *


from importlib import reload
%load_ext autoreload
%autoreload 2

In [ ]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--subject_id', metavar='subject_id', help='subject_id',
                    default='sub101')

# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/raw/QA/', 
                          '--output_folder', '../../data/QA/',
                          '--subject_id', 'sub103',
                         ])
# args = parser.parse_args()
print(args)

In [ ]:
inputdir = args.input_folder
outputdir = args.output_folder
subject_id = args.subject_id
# position = args.position

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
    
outputdir_sub = outputdir+subject_id+'/'

if not os.path.exists(outputdir_sub):
    os.makedirs(outputdir_sub)

In [ ]:
outputdir_sub

# read data from sternum, clavicle, and ribcage

In [ ]:
raw_dict = {}

for sub_file in os.listdir(inputdir):
    print(sub_file)
    if 'bin' not in sub_file:
        continue
        
#     print(sub_file.split('_'))
    patch_id = sub_file.split('_')[0]
    test_id = sub_file.split('_')[3].split('.')[0]
#     if sub_file.split('_')[0] != subject_id:
#         continue
#     for data_file in os.listdir(inputdir+sub_file):
    print(patch_id, test_id)

    if 'T10' not in sub_file:
        continue
    
#     patch_dict = {}
    
#     for 
#         patch_id = 
#         raw_dict[]
# #         if position in data_file:
# #         if position in list_positions: 
#         for position in list_positions:
#             if position in data_file:
#                 print('\n', position)
#                 fileName = inputdir+sub_file+'/'+data_file
#                 print(fileName)
    fileName = inputdir+sub_file
    file = open(fileName, mode='rb')
    fileContent = file.read()
    
    file_id = patch_id + '_' + test_id
    raw_dict[file_id] = parse(fileContent, header = True, print_updates = True, isConvUnits = True)
    raw_dict[file_id]['subject_id'] = subject_id

In [ ]:
# plt.plot(raw_dict['sternum']['ecg_time'][50000:50500], raw_dict['sternum']['ecg'][50000:50500])

# Do interpolation

In [ ]:
patch_dict = {}
df_patch = {}
for file_id in raw_dict.keys():
    print(file_id)
    patch_dict = data_interpolation(raw_dict[file_id], FS_RESAMPLE=FS_RESAMPLE)
    df_patch[file_id] = pd.DataFrame(patch_dict)
    df_patch[file_id]['time'] = df_patch[file_id]['time'] - df_patch[file_id]['time'].min()

In [ ]:
file_id = 'HP-0202_T9'
file_id = 'HP-0238_T9'
file_id = 'HP-0235_T9'
file_id = 'HP-0251_T8'

file_id = 'HP-0237_T10'
file_id = 'HP-0273_T10'

# inspect the taps

In [ ]:
# with open('patch_tap_times.json', 'r') as f:
#     patch_tap_times = json.load(f)[0]
    
inspect_plot = False
# if subject_id not in patch_tap_times:
if inspect_plot:
# Create traces
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    df = df_patch[file_id].copy()

    df = get_filt_dict(df)
#     t_start = df['time'].max() - 1000
# #     t_start = df_patch['time'].max()-20*60
#     t_end = df['time'].max()-500
    
    t_dur = df.shape[0]/FS_RESAMPLE
    t_start = 2
    t_end = t_dur-2
            
            
    df = df[(df['time']>t_start) & (df['time']<t_end)] # ignore the last 60minutes because the study is at least 60 minute long prior to stop recording
    downsample_factor = 1

    alpha = 0.5

    v_ECG = df['ECG'].values
    v_accelX = df['accelX'].values
    v_accelY = df['accelY'].values
    v_accelZ = df['accelZ'].values
    
    ts = df['time'].values
    
    
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ECG[::downsample_factor],
                        mode='lines',
                        name='ecg',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ECG']])),
                  secondary_y=True,)

    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_accelX[::downsample_factor],
                        mode='lines',
                        name='accelX',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['accelX']])),
                  secondary_y=False,)

    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_accelY[::downsample_factor],
                        mode='lines',
                        name='accelY',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['accelY']])),
                  secondary_y=False,)

    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_accelZ[::downsample_factor],
                        mode='lines',
                        name='accelZ',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['accelZ']])),
                  secondary_y=False,)

    # Add figure title
    fig.update_layout(
        title_text="inspect taps"
    )

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>signal (a.u.)</b>", secondary_y=False)

    fig.update_xaxes(title_text="<b>time (sec)</b>")
    fig.update_layout(plot_bgcolor='rgb(255,255,255)')

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)


    fig.show()
#     sys.exit()
else:
    tap_time = patch_tap_times[subject_id]
    print(tap_time)


In [ ]:
# df_patch['HP-0273_T10']

In [ ]:
# with open('patch_tap_times.json', 'r') as f:
#     patch_tap_times = json.load(f)[0]
    
inspect_plot = True
# if subject_id not in patch_tap_times:
if inspect_plot:
# Create traces
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    df = df_patch[file_id].copy()

    df = get_filt_dict(df)
#     t_start = df['time'].max() - 1000
# #     t_start = df_patch['time'].max()-20*60
#     t_end = df['time'].max()-500
    
    t_dur = df.shape[0]/FS_RESAMPLE
    t_start = 2
    t_end = t_dur-2
            
    df = df[(df['time']>t_start) & (df['time']<t_end)] # ignore the last 60minutes because the study is at least 60 minute long prior to stop recording
    downsample_factor = 1

    alpha = 0.5

    v_ECG = df['ECG'].values
    v_ppg_g_1 = df['ppg_g_1'].values
    v_ppg_r_1 = df['ppg_r_1'].values
    v_ppg_ir_1 = df['ppg_ir_1'].values
    v_ppg_g_2 = df['ppg_g_2'].values
    v_ppg_r_2 = df['ppg_r_2'].values
    v_ppg_ir_2 = df['ppg_ir_2'].values
    
    ts = df['time'].values
    
    
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ECG[::downsample_factor],
                        mode='lines',
                        name='ecg',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ECG']])),
                  secondary_y=True,)

    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_g_1[::downsample_factor],
                        mode='lines',
                        name='ppg_g_1',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_g_1']])),
                  secondary_y=False,)
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_r_1[::downsample_factor],
                        mode='lines',
                        name='ppg_r_1',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_r_1']])),
                  secondary_y=False,)
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_ir_1[::downsample_factor],
                        mode='lines',
                        name='ppg_ir_1',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_ir_1']])),
                  secondary_y=False,)

    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_g_2[::downsample_factor],
                        mode='lines',
                        name='ppg_g_2',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_g_2']])),
                  secondary_y=False,)
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_r_2[::downsample_factor],
                        mode='lines',
                        name='ppg_r_2',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_r_2']])),
                  secondary_y=False,)
    fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_ir_2[::downsample_factor],
                        mode='lines',
                        name='ppg_ir_2',
                             opacity=alpha,
                          line=dict(color=color_dict[sync_color_dict['ppg_ir_2']])),
                  secondary_y=False,)


    # Add figure title
    fig.update_layout(
        title_text="inspect taps"
    )

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>signal (a.u.)</b>", secondary_y=False)

    fig.update_xaxes(title_text="<b>time (sec)</b>")
    fig.update_layout(plot_bgcolor='rgb(255,255,255)')

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)


    fig.show()
#     sys.exit()
else:
    tap_time = patch_tap_times[subject_id]
    print(tap_time)


# checklist 1: find out sampling rate for each sensor
ECG: 499.01 Hz 
ACC: 1000.95 Hz 
PPG: 67.04 Hz 

In [ ]:
# for file_id in raw_dict.keys():
#     print('\nchecking:', file_id)
#     T_ecg = np.diff(raw_dict[file_id]['ecg_time'])
#     T_accel = np.diff(raw_dict[file_id]['accel_time'])
#     T_ppg = np.diff(raw_dict[file_id]['ppg_time'])

#     print('\tECG: {:.2f} Hz '.format(1/T_ecg.mean()))
#     print('\tACC: {:.2f} Hz '.format(1/T_accel.mean()))
#     print('\tPPG: {:.2f} Hz '.format(1/T_ppg.mean()))

# checklist 2: take a look at the signals
You should see three distinctive taps near t = 0 sec

In [ ]:
# FILT_PPG = [0.1, 4]
# FILT_ECG = [10, 30]
# # FILT_ECG = [1, 30]
# # FILT_SCG = [1, 25]
# FILT_SCG = [5, 25]
get_filt_dict

In [ ]:
make_some_plot = True

if make_some_plot:
    for file_id in df_patch.keys():
#         print(file_id)
#         print(file_id.split('_')[1])

#         if file_id.split('_')[1]!='T6':
#             continue
        
        if file_id.split('_')[1]=='T4':
            t_end = df_patch[file_id].shape[0]/FS_RESAMPLE
    #         print(file_id, t_end)
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_end-6, VIS_END=t_end-2, outputdir=outputdir_sub+file_id.split('_')[1]+'/', show_plot=False)

        if 'T6' in file_id.split('_')[1]:
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = t_dur-20
            t_end = t_dur+20
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T2' in file_id.split('_')[1]:
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = t_dur-15
            t_end = t_dur+15
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T3' in file_id.split('_')[1]:
                    t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
                    t_start = t_dur-20
                    t_end = t_dur+20
                    inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T7' in file_id.split('_')[1]:
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = t_dur-20
            t_end = t_dur+20
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T5' in file_id.split('_')[1]:
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = t_dur-11
            t_end = t_dur-10.5
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
#             inspect_patch(df_patch[file_id], title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T8' in file_id.split('_')[1]:
#             print(file_id.split('_')[1])
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = 8
            t_end = t_dur-8
#             print(df_patch[file_id])
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end,plt_scale=50/3, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
#             inspect_patch(df_patch[file_id], title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T9' in file_id.split('_')[1]:
#             print(file_id.split('_')[1])
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = 8
            t_end = t_dur-8
#             print(df_patch[file_id])
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end,plt_scale=50/3, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
#             inspect_patch(df_patch[file_id], title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        if 'T10' == file_id.split('_')[1]:
#             print(file_id.split('_')[1])
            t_dur = df_patch[file_id].shape[0]/FS_RESAMPLE
            t_start = t_dur-60
            t_end = t_dur-10
#             print(df_patch[file_id])
            inspect_patch(get_filt_dict(df_patch[file_id]), title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end,plt_scale=50/10, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
#             inspect_patch(df_patch[file_id], title_str='filt_'+file_id, VIS_START=t_start, VIS_END=t_end, outputdir=outputdir_sub+file_id.split('_')[1][:2]+'/', show_plot=False)
        

In [ ]:
outputdir_sub+file_id.split('_')[1][:2]+'/'

In [ ]:
file_id.split('_')[1]